# Ensembles with Tensorflow and xgb

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys

import copy
import csv
import datetime
import itertools
import numpy as np
import pandas as pd
import random
import re
import sklearn
import time
from collections import defaultdict

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context(rc={
       "figure.figsize": (16, 10),
       "axes.titlesize": 14})

from IPython.display import Image, display
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

from os.path import expanduser
sys.path.insert(1, '{}/datsci'.format(expanduser('~')))
from datsci import eda, munge, ml, ensemble
from datsci import kaggle as kg

In [2]:
import santander

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier as ABC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.linear_model import SGDClassifier as SGDClf
from sklearn.cross_validation import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import make_scorer

import xgboost as xgb

In [ ]:
import skflow

In [4]:
# Read in data
X_train, y_train, X_valid, y_valid, feature_cols, df_train, df_test = santander.read_split(
    santander.FILE_TRAIN_DEDUP_VAR3_DELTA1_1HOT_1HOTINT,
    santander.FILE_TEST_DEDUP_VAR3_DELTA1_1HOT_1HOTINT,
    test_size=0.1,
    random_state=santander.RANDOM_SEED
)

# 28 balanced subsets
bss = munge.balanced_subsets(X_train, y_train, subsample=0.85)

In [ ]:
# Get top features from xgb model

def select_features(X_train, y_train, X_valid, y_valid):
    model = xgb.XGBRegressor(
        learning_rate=0.01,
        n_estimators=5000,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        colsample_bylevel=0.8,
        objective='binary:logistic',
        nthread=4,
        seed=santander.RANDOM_SEED
    )
    n_estimators, train_score, test_score = ml.cv_fit_xgb_model(
        model, X_train, y_train, X_valid, y_valid, cv_nfold=5, early_stopping_rounds=50,
        scoring="roc_auc", verbose=False
    )

    # Get important features
    return list(pd.Series(model.booster().get_fscore()).sort_values(ascending=False).index)

In [ ]:
X, y = bss[0]

model_nn = skflow.TensorFlowDNNClassifier(
    hidden_units=[100], 
    n_classes=2, 
    batch_size=200,
    steps=3000, 
    learning_rate=0.1)

classifier.fit(X_train_scaled, y_train)